<hr style="height:2px;">

# Demo: Training data generation for denoising of *Tribolium castaneum*

### Notes

- Training data generation for the case where both low SNR and corresponding high SNR stacks can be acquired.
- Documentation available: http://csbdeep.bioimagecomputing.com/doc/

In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
from tifffile import imread
from csbdeep import datagen
from csbdeep.plot_utils import plot_some

<hr style="height:2px;">

# Download example data

In [ ]:
from csbdeep.utils import download_and_extract_zip_file
download_and_extract_zip_file(
    url = 'https://cloud.mpi-cbg.de/index.php/s/jKHFIS4isNwagMd/download',
    provides = ('raw_data/tribolium/%s/nGFP_0.1_0.2_0.5_20_13_late.tif'%d for d in ('GT','low'))
)

Data should have been downloaded like this:

    raw_data/tribolium
    ├── GT
    │   └── nGFP_0.1_0.2_0.5_20_13_late.tif
    └── low
        └── nGFP_0.1_0.2_0.5_20_13_late.tif

<hr style="height:2px;">

# Raw data

Corresponding low and high SNR stacks should have identical names but located in different folders.  
We only show one stack for demonstration, but you should have ~20-50 different stacks at different developmental timepoints.

Let's just plot the low and high (GT) stacks:

In [ ]:
y = imread('raw_data/tribolium/GT/nGFP_0.1_0.2_0.5_20_13_late.tif')
x = imread('raw_data/tribolium/low/nGFP_0.1_0.2_0.5_20_13_late.tif')
axes = 'ZYX'
print('image size =', x.shape)
print('image axes =', axes)

plt.figure(figsize=(15,10))
plot_some(np.stack([x,y]),title_list=[['low (maximum projection)','GT (maximum projection)']], pmin=2,pmax=99.8);

<hr style="height:2px;">

# Patch generation

From corresponding stacks, we generate 3D patches of size (16,64,64) that are sampled from regions of interesting structures. 

In [ ]:
raw_data = datagen.get_tiff_pairs_from_folders (
    basepath    = 'raw_data/tribolium',
    source_dirs = ['low'],
    target_dir  = 'GT',
    axes        = axes,
)

X, Y, XY_axes = datagen.create_patches (
    raw_data            = raw_data,
    patch_size          = (16,64,64),
    n_patches_per_image = 1024,
    save_file           = 'my_training_data.npz',
)

In [ ]:
assert X.shape == Y.shape
print("shape of X,Y =", X.shape)
print("axes  of X,Y =", XY_axes)

## Show

This shows the maximum projection of some of those patch pairs (first row: *source*, second row: *target*)

In [ ]:
from csbdeep.plot_utils import plot_some

for i in range(2):
    plt.figure(figsize=(16,4))
    sl = slice(8*i, 8*(i+1)), 0
    plot_some(X[sl],Y[sl],title_list=[np.arange(sl[0].start,sl[0].stop)])
    plt.show()
None;